# Assignment
- Learn about the mathematics of Logistic Regression by watching Aaron Gallant's [video #1](https://www.youtube.com/watch?v=pREaWFli-5I) (12 minutes) & [video #2](https://www.youtube.com/watch?v=bDQgVt4hFgY) (9 minutes).
- Start a clean notebook.
- Do train/validate/test split with the Tanzania Waterpumps data.
- Begin to explore and clean the data. For ideas, refer to [The Quartz guide to bad data](https://github.com/Quartz/bad-data-guide),  a "reference to problems seen in real-world data along with suggestions on how to resolve them." One of the issues is ["Zeros replace missing values."](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values)
- Select different numeric and categorical features. 
- Do one-hot encoding. (Remember it may not work with high cardinality categoricals.)
- Scale features.
- Use scikit-learn for logistic regression.
- Get your validation accuracy score.
- Get and plot your coefficients.
- Submit your predictions to our Kaggle competition.
- Commit your notebook to your fork of the GitHub repo.

## Stretch Goals
- Begin to visualize the data.
- Try different [scikit-learn scalers](https://scikit-learn.org/stable/modules/preprocessing.html)
- Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html):

> Pipeline can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification. Pipeline serves multiple purposes here:

> - **Convenience and encapsulation.** You only have to call fit and predict once on your data to fit a whole sequence of estimators.
> - **Joint parameter selection.** You can grid search over parameters of all estimators in the pipeline at once.
> - **Safety.** Pipelines help avoid leaking statistics from your test data into the trained model in cross-validation, by ensuring that the same samples are used to train the transformers and predictors.

In [38]:
import pandas as pd
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
# !kaggle competitions download -c ds4-predictive-modeling-challenge

In [3]:
# !unzip test_features.csv.zip
# !unzip train_features.csv.zip
# !unzip train_labels.csv.zip

In [4]:
# !ls

In [44]:
# csv files were saved by kaggle with no read or write permissions?
train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')
test_features = pd.read_csv('test_features.csv')
train_features.shape, train_labels.shape, test_features.shape

((59400, 40), (59400, 2), (14358, 40))

In [45]:
train_features.describe(include='all')


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,59400.00,59400.00,59400,55765,59400.00,55745,59400.00,59400.00,59400,59400.00,59400,59029,59400,59400.00,59400.00,59400,59400,59400.00,56066,59400,55523,31234,56344,59400.00,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,nan,nan,356,1897,nan,2145,nan,nan,37400,nan,9,19287,21,nan,nan,125,2092,nan,2,1,12,2696,2,nan,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6
top,nan,nan,2011-03-15,Government Of Tanzania,nan,DWE,nan,nan,none,nan,Lake Victoria,Madukani,Iringa,nan,nan,Njombe,Igosi,nan,True,GeoData Consultants Ltd,VWC,K,True,nan,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,nan,nan,572,9084,nan,17402,nan,nan,3563,nan,10248,508,5294,nan,nan,2503,307,nan,51011,59400,36793,682,38852,nan,26780,26780,26780,40507,52490,25348,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625
mean,37115.13,317.65,NaN,NaN,668.30,NaN,34.08,-5.71,NaN,0.47,NaN,NaN,NaN,15.30,5.63,NaN,NaN,179.91,NaN,NaN,NaN,NaN,NaN,1300.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,21453.13,2997.57,NaN,NaN,693.12,NaN,6.57,2.95,NaN,12.24,NaN,NaN,NaN,17.59,9.63,NaN,NaN,471.48,NaN,NaN,NaN,NaN,NaN,951.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.00,0.00,NaN,NaN,-90.00,NaN,0.00,-11.65,NaN,0.00,NaN,NaN,NaN,1.00,0.00,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,18519.75,0.00,NaN,NaN,0.00,NaN,33.09,-8.54,NaN,0.00,NaN,NaN,NaN,5.00,2.00,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,37061.50,0.00,NaN,NaN,369.00,NaN,34.91,-5.02,NaN,0.00,NaN,NaN,NaN,12.00,3.00,NaN,NaN,25.00,NaN,NaN,NaN,NaN,NaN,1986.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,55656.50,20.00,NaN,NaN,1319.25,NaN,37.18,-3.33,NaN,0.00,NaN,NaN,NaN,17.00,5.00,NaN,NaN,215.00,NaN,NaN,NaN,NaN,NaN,2004.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
train_labels['status_group']

0                     functional
1                     functional
2                     functional
3                 non functional
4                     functional
5                     functional
6                 non functional
7                 non functional
8                 non functional
9                     functional
10                    functional
11                    functional
12                    functional
13                    functional
14                    functional
15                    functional
16                non functional
17                non functional
18       functional needs repair
19                    functional
20                    functional
21                    functional
22       functional needs repair
23                    functional
24                    functional
25       functional needs repair
26                    functional
27                    functional
28                non functional
29                    functional
30        

In [8]:
def return_mean_if_zero(data, column):
    if data == 0:
        return column.mean()
    else:
        return data

def not_the_right_way(data, column):
    if data == -2e-08:
        return column.mean()
    else:
        return data
    
train_features['longitude'] = train_features['longitude'].apply(return_mean_if_zero, args=(train_features['longitude'],))
train_features['latitude'] = train_features['latitude'].apply(not_the_right_way, args=(train_features['latitude'],))

In [9]:
# train_features = train_features.drop(['recorded_by'], axis=1)
dummied_features = pd.get_dummies(train_features, columns=['management_group', 'payment_type', 'source_type', 'water_quality'],
               prefix=['mgmt', 'payment', 'source', 'quality'])

In [47]:
train_features.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
recorded_by,59400,1,GeoData Consultants Ltd,59400
public_meeting,56066,2,True,51011
permit,56344,2,True,38852
source_class,59400,3,groundwater,45794
management_group,59400,5,user-group,52490
quantity_group,59400,5,enough,33186
quantity,59400,5,enough,33186
waterpoint_type_group,59400,6,communal standpipe,34625
quality_group,59400,6,good,50818
payment_type,59400,7,never pay,25348


In [11]:


# Returns X_train, X_val, y_train, y_val
def quick_split(X, y):
    X_train = X
    y_train = y

    return train_test_split(
        X_train, y_train, train_size=0.80, test_size=0.20,
        stratify=y_train)

In [60]:
def fit_predict_score(X, y, X_val, y_val):
    model = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=20000)
    model.fit(X, y)
    y_pred = model.predict(X_val)
    print(model.coef_)
#     sample_submission = pd.read_csv('sample_submission.csv')
#     submission = sample_submission.copy()
#     submission['status_group'] = y_pred
#     submission.to_csv('whaeck-submission.csv', index=False)
    return accuracy_score(y_val, y_pred)

In [15]:
def fit_submission(X, y, ):
    pass

SyntaxError: invalid syntax (<ipython-input-15-439abf10002e>, line 1)

In [24]:
dummied_features = pd.get_dummies(train_features, columns=['management_group', 'quality_group', 'waterpoint_type', 'extraction_type_group', 'basin', 'public_meeting', 'permit'],
               prefix=['mgmt', 'quality', 'waterpoint', 'extraction', 'basin', 'public', 'permit'])

In [ ]:
# dummied_features['public_meeting']
# dummied_features['public_meeting'] = dummied_features['public_meeting'].astype('category')
# dummied_features['permit'] = dummied_features['permit'].astype('category')
# dummied_features['public_meeting_cat'] = dummied_features['public_meeting'].cat.codes
# dummied_features['permit_cat'] = dummied_features['permit'].cat.codes

In [26]:
dummied_features = dummied_features.drop('id', axis=1)

In [27]:
X_train, X_val, y_train, y_val = quick_split(dummied_features.select_dtypes('number'), train_labels['status_group'])
fit_predict_score(X_train, y_train, X_val, y_val)

0.5507575757575758

In [128]:
encoder = ce.OneHotEncoder(use_cat_names=True)
categorical_features = ['management_group', 'payment_type', 'source_class', 'quality_group', 'quantity', 'waterpoint_type_group', 'extraction_type_group', 'basin', 'source', 'region', 'public_meeting', 'permit', 'installer', 'ward']
numeric_features = train_features.select_dtypes('number').columns.drop('id').tolist()
features = categorical_features + numeric_features
encoded = encoder.fit_transform(train_features[features])

In [130]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(encoded)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [131]:
X_train, X_val, y_train, y_val = quick_split(X_scaled, train_labels['status_group'])
# fit_predict_score(X_train, y_train, X_val, y_val)

In [132]:
model = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=20000)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
print(model.coef_)
accuracy_score(y_val, y_pred)

[[-0.07449389  0.01676034  0.09190416 ...  0.27285962  0.09108923
   0.05725912]
 [ 0.04310237 -0.02514243 -0.11125997 ... -0.12792543  0.02619924
  -0.18015363]
 [ 0.03139153  0.0083821   0.01935581 ... -0.14493418 -0.11728847
   0.12289451]]


0.7707070707070707

In [139]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10), dpi=1200)
coefficients = pd.Series(model.coef_[0], encoded.columns)
coefficients.sort_values().plot.barh();

In [134]:
categorical_features = ['management_group', 'payment_type', 'source_class', 'quality_group', 'quantity', 'waterpoint_type_group', 'extraction_type_group', 'basin', 'source', 'region', 'public_meeting', 'permit', 'installer', 'ward']
numeric_features = test_features.select_dtypes('number').columns.drop('id').tolist()
features = categorical_features + numeric_features
test_encoded = encoder.transform(test_features[features])

X_test_scaled = scaler.transform(test_encoded)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [135]:
y_pred = model.predict(X_test_scaled)
submission = pd.read_csv('sample_submission.csv')
submission['status_group'] = y_pred
submission.to_csv('submission-01.csv', index=False)

In [137]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=400, n_jobs=-1)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [138]:
y_pred = rfc.predict(X_val)
accuracy_score(y_val, y_pred)

0.8047979797979798

# Pipelines
## Isn't it enough to know I ruined a pony to make a gift for you?

In [ ]:
from sklearn.pipeline import Pipeline